In [1]:
from train_arcface import train_arcface
from src.utils.model import get_triplet_model, freeze_layer
from src.model.arcface import KSubArcFace, ArcFace_org
import torch
from train_triplet import train_triplet
from train_arc_triplet import train_arc_triplet


from torch.utils.tensorboard import SummaryWriter

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
WEIGHT_PATH = 'weight/backbone_r100_glint360k.pt'
TRAIN_PATH = "Data/train_triplet_processed.txt"
TEST_PATH = "Data/train_triplet_processed.txt"
EPOCHS = 10
BATCH__SIZE = 64

# FINETUNE_PATH = 'weight/model_finetune_arcface_org_best.pth'
ARCFACE_PATH = 'weight/arcface_epoch=66500_val_F1=0.9624_last.pth'

In [3]:
model = get_triplet_model(weight_path= WEIGHT_PATH)
# model.backbone.load_state_dict(torch.load(FINETUNE_PATH, map_location = 'cpu')['model_state_dict'])

arcface = ArcFace_org(in_features = 512, out_features = 18948, s = 30, m = 1.)
arcface.load_state_dict(torch.load(ARCFACE_PATH, map_location = 'cpu')['model_checkpoint'])


train_arcface(model, arcface, TRAIN_PATH, TEST_PATH, EPOCHS, BATCH__SIZE, checkpoint_path= 'weight_arcface_10_7' )

OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'runs/arcface_s=30_m=1.0_2023-07-10 16:16:25.658359'

In [ ]:
# model = get_triplet_model(weight_path= WEIGHT_PATH)
# freeze_layer(model, ['253', '254'])
# train_triplet(model, TRAIN_PATH, TEST_PATH, EPOCHS, BATCH__SIZE, roc_fig_path='result/finetune_triplet_loss.jpg')

In [1]:
from src.data import TripletFaceDataset, Triplet_loader 
import torch
test_dataset= Triplet_loader("Data/valid_triplet_processed.txt")
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=int(.75 * 64),
                                        shuffle=False, num_workers= 0, drop_last = True)

In [ ]:
from src.data import TripletFaceDataset, Triplet_loader
from src.utils.model import get_test_model

val_model = get_test_model(weight_path= None,  init_weight = "weight/backbone_r100_glint360k.pt")
val_model.cuda()

In [5]:
from test import validation
import torch.nn as nn
import torch.nn.functional as F

def sim_loss(x1,x2):
    return torch.clamp_min(1 - F.cosine_similarity(x1,x2),0)

validation(val_model,  test_loader = testloader, criterion =  nn.TripletMarginWithDistanceLoss(distance_function = sim_loss, margin=1.0)
)

100%|██████████| 182/182 [02:08<00:00,  1.41it/s]


Best threshold: 0.3140852451324463
precision: 0.9752702951330642
recall: 0.967054429945055
F1 score: 0.9711449863403436
Accuracy: 0.9712665264423077


(0.9711449863403436, 0.4558490773478707)